# # joblib
---

In [3]:
import os
os.cpu_count()

48

In [ ]:
from joblib import Parallel, delayed
import os

# csv파일과 json파일을 각각 리스트로 가져옴
csv_files8, json_files8 = Parallel(os.cpu_count())(delayed(V2XPreprocessing.get_csv_data)(data_path + "/CSV"), \
                            V2XPreprocessing.get_json_data(data_path + "/JSON")

# csv파일과 json파일 리스트를 pickle로 저장
# with open(f"{save_path}/CSV8.pkl", "wb") as f:
#     pkl.dump(csv_files8, f)
# with open(f"{save_path}/JSON8.pkl", "wb") as f:
#     pkl.dump(json_files8, f)

print(f"csv file count: {len(csv_files8)}, json file count: {len(json_files8)}")

In [ ]:
def get_csv_data(csv_folder: str) -> List[str]:
    from joblib import Parallel, delayed
    with Parrallel(n_jobs=os.cpu_count()) as prl:
        csv_files = prl(delayed(([str(p) for p in pl.Path(csv_folder).rglob("*.csv")])
    return csv_files

Parallel(n_jobs=2)(delayed(V2XPreprocessing.get_csv_data)(data_path + "/CSV"))

# # ProcessPoolExecutor
---

In [5]:
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from functools import partial
from tqdm import tqdm

## 1. 배열 자르기
to_work_array = np.arange(1e5, dtype=np.int32)
num_processes = 6
batch_size = len(to_work_array)//num_processes
test = 0

# 배열을 중복생성하지 않고 인덱스만 저장
# start_end_idxs = []
# for i in range(num_processes+1):
#     start_end_idxs.append(
#         {
#             's':batch_size*i,
#             'e':batch_size*(i+1)
#         }
#     )
start_end_idxs = [{'s': batch_size*i, 'e': batch_size*(i+1)} for i in np.arange(num_processes+1)]

# 2. 함수 정의
def compute_array(array, num):
    # global test
    # test = 10
    return [item/2 for item in tqdm(array)]


## 3. 병렬 실행
futures = []
results = []
with ProcessPoolExecutor(num_processes) as executor:
    for idx in start_end_idxs:
        batch = to_work_array[idx['s']:idx['e']]

        task = partial(compute_array, batch, 1)
        future = executor.submit(task)
        futures.append(future)

        # result = executor.submit(task).result() # 바로 뽑으면 멀티 프로세싱이 실행이 안된다
        # results.extend(result)

    for future in futures:
        chunk_result = future.result()
        results.extend(chunk_result)

100%|██████████| 4/4 [00:00<00:00, 26379.27it/s]


In [48]:
import os
import time
from concurrent import futures


WORK_LIST = [100000, 1000000, 10000000, 100000000]

# 동시성 합계 계산 메인함수
# 누적 합계 함수(제네레이터)
def sum_generator(n):
    return sum(n for n in tqdm(range(1, n+1)))

def main():
    # Worker Count
    worker = min(4, len(WORK_LIST))
    # 시작 시간
    start_tm = time.time()
    # 결과 건수
    # ProcessPoolExecutor
    # with futures.ThreadPoolExecutor() as excutor:
    with futures.ProcessPoolExecutor(worker) as excutor:
        # map -> 작업 순서 유지, 즉시 실행, 리턴타입 제너레이터
        result = excutor.map(sum_generator, WORK_LIST)

    print('type(result)', type(result))
    
    # 종료 시간
    end_tm = time.time() - start_tm
    # 출력 포멧
    msg = '\n Result -> {} Time : {:05.2f}s'
    # 최종 결과 출력
    print(msg.format(list(result), end_tm))

# 실행
if __name__ == '__main__':
    main()

100%|██████████| 100000000/100000000 [00:15<00:00, 6614402.59it/s]


type(result) <class 'generator'>

 Result -> [5000050000, 500000500000, 50000005000000, 5000000050000000] Time : 15.19s


In [1]:
import os
import time
from concurrent import futures
from tqdm import tqdm
import numpy as np

# WORK_LIST = [100000, 1000000, 10000000, 100000000]
WORK_LIST = tqdm(np.arange(1, 100))

# 동시성 합계 계산 메인함수
# 누적 합계 함수(제네레이터)
def sum_generator(n):

    def make_file(n):
        with open(f"test/{n}.txt", "w") as f:
            f.write('test')

    make_file(n)

    # for i in n:
    #     make_file(i)


def main():
    # Worker Count
    worker = min(10, len(WORK_LIST))
    # 시작 시간
    start_tm = time.time()
    # 결과 건수
    # ProcessPoolExecutor
    # with futures.ThreadPoolExecutor() as excutor:
    with futures.ProcessPoolExecutor(worker) as excutor:
        # map -> 작업 순서 유지, 즉시 실행, 리턴타입 제너레이터
        excutor.map(sum_generator, WORK_LIST)

    # print('type(result)', type(result))
    
    # 종료 시간
    end_tm = time.time() - start_tm
    # 출력 포멧
    msg = '\n Result -> {} Time : {:05.2f}s'
    # 최종 결과 출력
    # print(msg.format(list(result), end_tm))

# 실행
if __name__ == '__main__':
    main()

100%|██████████| 99/99 [00:00<00:00, 436.14it/s]


In [ ]:
from time import time
from concurrent.futures import ProcessPoolExecutor


def gcd(pair):
    a, b = pair
    low = min(a, b)
    for i in range(low, 0, -1):
        if a % i == 0 and b % i == 0:
            return i


# 병렬성이 없으므로 이 함수를 순서대로 실행하면 시간이 선형적으로 증가
numbers = [(1963309, 2265973), (2030677, 3814172), (1551645, 2229620),
           (2039045, 2020802)]
start = time()
pool = ProcessPoolExecutor(max_workers=4)
results = list(pool.map(gcd, numbers))
end = time()
print('Took %.3f seconds' % (end - start))

# # multiprocessing
---

In [14]:
from multiprocessing import Process, Queue
import time


def work():
    n = 0
    for i in range(10000000):
        n += 1
    time.sleep(0.1)


if __name__ == '__main__':
    # 싱글 프로세스
    start_time = time.time()
    for i in range(20):
        work()

    print(f"싱글 프로세스: {time.time() - start_time}")

    # 멀티 프로세스
    result = Queue()
    processes = []
    start_time = time.time()
    for i in range(20):
        processes.append(Process(target=work))
        processes[-1].start()

    while processes:
        processes.pop().join()

    print(f"멀티 프로세스: {time.time() - start_time}")

싱글 프로세스: 10.360847473144531
멀티 프로세스: 0.8460934162139893


In [83]:
import time


def heavy_work(name):
    result = 0
    for i in tqdm(range(4000000)):
        result += i
    print('%s done' % name)


if __name__ == '__main__':
    import multiprocessing

    start = time.time()
    procs = []
    for i in range(4):
        p = multiprocessing.Process(target=heavy_work, args=(i, ))
        p.start()
        procs.append(p)

    for p in procs:
        p.join()  # 프로세스가 모두 종료될 때까지 대기

    end = time.time()

    print("수행시간: %f 초" % (end - start))

100%|██████████| 4000000/4000000 [00:00<00:00, 5856802.89it/s]


0 done


100%|██████████| 4000000/4000000 [00:00<00:00, 5430941.63it/s]


1 done


100%|██████████| 4000000/4000000 [00:00<00:00, 5460115.83it/s]


3 done


100%|██████████| 4000000/4000000 [00:00<00:00, 4705124.02it/s]


2 done
수행시간: 0.897320 초


In [21]:
import time
import numpy as np
from tqdm.auto import tqdm


def heavy_work(name):
    result = 0
    for i in tqdm(range(4000000)):
        result += i
    print('%s done' % name)


if __name__ == '__main__':
    import multiprocessing

    start = time.time()
    pool = multiprocessing.Pool(processes=20)
    pool.map(heavy_work, np.arange(7))
    pool.close()
    pool.join()

    end = time.time()

    print("수행시간: %f 초" % (end - start))

4 done5 done

1 done
3 done
0 done
6 done
2 done
수행시간: 2.472358 초


In [20]:
# multi argument multiprocessing 1

import time
import numpy as np
from tqdm.auto import tqdm


def heavy_work(name, nothing):
    result = []
    for i in tqdm(range(1000)):
        result.append(i)
    return result
    print('%s done' % name)


if __name__ == '__main__':
    import multiprocessing

    start = time.time()
    pool = multiprocessing.Pool(processes=20)
    pool.starmap(heavy_work, zip(np.arange(20), np.arange(20)))
    pool.close()
    pool.join()

    end = time.time()

    print("수행시간: %f 초" % (end - start))

수행시간: 2.167622 초


In [25]:
# multi argument multiprocessing 2

import multiprocessing
from functools import partial

# 실행하고자 하는 함수의 인자가 하나
def foo(x):
    return x

pool = multiprocessing.Pool(processes=4)
result = pool.map(foo, range(10))
pool.close()
pool.join()
result


# 실행하고자 하는 함수의 인자가 여러개
def foo(x, y):
    return x*y

pool = multiprocessing.Pool(processes=4)
func = partial(foo, y=2)
result = pool.map(func, range(10))
pool.close()
pool.join()
result

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]